In [ ]:
%sql

/*1. Filtrando apenas clientes da classe social que mais comprou produtos no período (em valor de venda), diga quantos clientes compraram peças em promoção em cada unidade da federação.*/

WITH ClasseSocialMaisComprou AS (
SELECT  clientes.ClasseSocial,
  ROUND(SUM(ROUND(CAST(REPLACE(vendas.ValueSold, ',', '.') AS FLOAT),2)),2) AS TotalVendas
FROM clientes clientes

LEFT JOIN vendas vendas
ON clientes.CustomerID = vendas.CustomerID

GROUP BY clientes.ClasseSocial
ORDER BY TotalVendas DESC
LIMIT 1
),

ClientesPromocaoPorUF AS (
SELECT
    lojas.UF,
    COUNT(DISTINCT clientes.CustomerID) AS TotalClientesPromo
FROM lojas

LEFT JOIN vendas
ON lojas.StoreID = vendas.StoreID

LEFT JOIN clientes
ON clientes.CustomerID = vendas.CustomerID

WHERE vendas.Promo = 1
AND clientes.ClasseSocial = (SELECT ClasseSocial FROM ClasseSocialMaisComprou)
GROUP BY lojas.UF
)
SELECT * FROM ClientesPromocaoPorUF
ORDER BY TotalClientesPromo DESC;



/*2. Filtrando apenas as transações de peças da cor azul realizadas por clientes que compraram peças da cor azul em apenas uma unidade da federação, diga qual foi o tamanho que mais vendeu peças.*/

WITH ClientesUnicoUF AS (
SELECT CustomerID
FROM  vendas v

LEFT JOIN lojas l
ON v.StoreID = l.StoreID

WHERE Cor = 'Azul'
GROUP BY CustomerID
HAVING COUNT(DISTINCT v.StoreID) = 1
)

SELECT
    LTRIM(v.Tamanho) AS Tamanho,
    COUNT(*) AS TotalVendas
FROM vendas v

JOIN ClientesUnicoUF cliuf
ON v.CustomerID = cliuf.CustomerID

WHERE Cor = 'Azul'
GROUP BY Tamanho
ORDER BY TotalVendas DESC
LIMIT 1;

/*3. Filtrando apenas o tipo de item (com base no campo Descrição) que vendeu mais peças da cor vermelha em 2023 do que em 2022 (em todas as UFs), qual foi o valor vendido no RS em 2022 para esse tipo de item (para peças de qualquer cor)?*/

WITH VendasAno AS(
SELECT `Descrição`,
        SUM(UnitsSold) as TotalVendas,
        YEAR(TO_DATE(v.`Date`, 'dd/MM/yyyy'))  AS Ano
FROM vendas v
WHERE Cor = 'Vermelho'
GROUP BY `Descrição`, Ano
),


ItemMaisVendido23 AS (
SELECT v23.`Descrição`
FROM VendasAno v23

JOIN VendasAno v22
ON v23.`Descrição` = v22.`Descrição`

WHERE v23.Ano = 2023
AND v22.Ano =2022
AND v23.TotalVendas > v22.TotalVendas
)


SELECT v.`Descrição`,
      l.UF,
       ROUND(SUM(ROUND(CAST(REPLACE(v.ValueSold, ',', '.') AS FLOAT),3)),3) AS TotalVendas
FROM vendas v

LEFT JOIN lojas l ON v.StoreID = l.StoreID

WHERE v.`Descrição` IN (SELECT `Descrição` FROM ItemMaisVendido23)
AND l.UF = 'RS'
AND YEAR(TO_DATE(v.`Date`, 'dd/MM/yyyy')) = '2022'
GROUP BY v.`Descrição`, l.UF


